In [ ]:
import matplotlib.dates as mdates
import mpl_panel_builder as mpb
import numpy as np
import pandas as pd
from mpl_panel_builder.helpers.mpl import adjust_axes_size

from loadshift import LoadShifter

# Use autoreload magic so that .py files can modified without having to
# restart the kernel
%load_ext autoreload
%autoreload 2

In [ ]:
data = pd.read_csv("../data/example_data.csv", index_col=0, parse_dates=True)

mpb_config = {
    "panel": {
        "dimensions": {"width_cm": 20, "height_cm": 6},
        "margins": {
            "left_cm": 0.75,
            "bottom_cm": 1.25,
            "right_cm": 0.25,
            "top_cm": 0,
        },
        "axes_separation": {"x_cm": 0.5, "y_cm": 0.75},
    },
    "style": {
        "theme": "presentation",
        "rc_params": {
            "figure.facecolor": "white",
            "axes.facecolor": "none",
            "legend.facecolor": "white",
            "font.size": 10,
        },
    },
    "output": {
        "format": "png",
        "dpi": "600"
    },
}

In [ ]:
red = "#EB6D44"
gray = "#9EADB2"
green = "#50AA46"
bar_width = 1/24*0.9
max_rate = 0.5
flexibilities = [0, 2, 4]
titles = [
    "No load shifting",
    f"Load shifting ±{flexibilities[1]} hour",
    f"Load shifting ±{flexibilities[2]} hours"
]

mpb.configure(mpb_config)
mpb.set_rc_style()
fig, axs = mpb.create_panel(rows=2, cols=3)

for col in range(3):
    ax = axs[0][col]
    adjust_axes_size(ax, length_cm=1, direction="top")
    ax.plot(data.index, data["price"], 'k-',solid_capstyle='round')
    ax.set(xticks=[], yticks=[], title=titles[col])
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    if col == 0:
        ax.set_ylabel("Price")

for col in range(3):

    flexivility = flexibilities[col]
    optimizer = LoadShifter(
        max_demand_advance=flexivility,
        max_demand_delay=flexivility,
        max_hourly_purchase=2*max_rate,
        max_rate=max_rate,
    )
    shift = optimizer.optimize_demand(
        price=data["price"].values, 
        demand=data["demand"].values
    )["optimal_shift"]
    remove = np.minimum(shift, 0)
    add = np.maximum(shift, 0)

    ax = axs[1][col]
    ax.bar(
        data.index,
        data["demand"].values,
        bar_width,
        fc=gray,
        label="Original demand",
    )
    ax.bar(
        data.index,
        add,
        bar_width,
        bottom=data["demand"].values,
        fc=green,
        label="Added demand",
    )
    ax.bar(
        data.index,
        -remove,
        bar_width,
        bottom=data["demand"].values+remove,
        fc=red,
        label="Removed demand",
    )
    
    ax.set(xlabel="Hour", yticks=[], ylim=[0, 2*max_rate])
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_linewidth(3)
    ax.spines['bottom'].set_capstyle('round')

    ax.xaxis.set_major_locator(mdates.HourLocator(interval=8))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

    if col == 0:
        ax.set_ylabel("Demand")
    if col == 1:
        ax.legend(
            loc="center",
            bbox_to_anchor=(0.5, 1),
            ncol=3
        )

mpb.save_panel(fig, "load_shift_example")